# Cross-Matching ZTF and Pan-STARRS using LSDB

[LSDB](https://lsdb.io) is a useful package for performing large cross-matches between source catalogs. It's built to run across multiple nodes with Dask parallelization, but even without parallelization it is high-performance. Here we will benchmark the performance of LSDB on the NASA Fornax platform with and without Dask.

We will start small, trying to cross-match 10,000 sources from ZTF with Pan-STARRS. We will then scale up by factors of roughly 10 until either (a) the platform can no longer handle the load, or (b) we do the full cross-match.

For each level, we want to know the performance with (1) no Dask, (2) minimal Dask - like 2 workers, (3) bigger Dask - as many workers as we can use, and (4) auto-scaling Dask.

## Preconfiguring the Run
First choose the number of rows we want to cross-match and our `dask` environment. Note that you can also configure `dask` using the `daskhub` options on Fornax. If you go this route, leave `dask_workers = 0` below.

In [ ]:
# The left table will have about this many rows. The cross-matched product will have fewer.
Nrows = 10_000 

# dask_workers can be 0 (no dask), 1-Ncores, or "scale" for auto-scaling
dask_workers = 0

## Imports
We require the use of `astropy` for coordinates and units, and `lsdb` to read the catalogs and perform the cross-match. Optionally, we will set up `dask` parallelization.

In [ ]:
from astropy.coordinates import SkyCoord
from astropy import units as u
from lsdb.core.search import ConeSearch

import lsdb

# Set up dask cluster
if dask_workers != 0:
    from dask.distributed import Client, LocalCluster
    cluster = LocalCluster()

    if dask_workers == "scale":
        import os
        cluster.adapt(minimum_cores=1, maximum_cores=os.cpu_count())
    else:
        cluster.scale(dask_workers)
        
    client = Client(cluster)
    client

# Select the search radius to give us the right number of rows
radius = { # Nrows: radius_arcseconds
           10_000:     331,
          100_000:    1047,
        1_000_000:    3318,
       10_000_000:  11_180,
      100_000_000:  33_743,
    1_000_000_000: 102_000, # runs out of memory on Large Fornax server
    # larger than that is effectively a full cross-match
}

## Read in catalogs and downselect ZTF to Nrows

In [ ]:
# Define sky area. Here we're using the Kepler field.
c = SkyCoord('19:22:40  +44:30:00', unit=(u.hourangle, u.deg))
cone_ra, cone_dec = c.ra.value, c.dec.value
radius_arcsec = radius[Nrows]
cone_filter = ConeSearch(cone_ra, cone_dec, radius_arcsec)

# Read ZTF DR20
ztf_path = ("s3://irsa-mast-tike-spitzer-data/data/ZTF/dr20/objects/hipscat/ztf-dr20-objects-hipscat")
ztf_piece = lsdb.read_hipscat(ztf_path, columns=["oid", "ra", "dec"], search_filter=cone_filter)

# Read Pan-STARRS DR2
ps1_path = "s3://stpubdata/panstarrs/ps1/public/hipscat/otmo"
ps1 = lsdb.read_hipscat(ps1_path, storage_options={'anon': True},
    columns=["objName","objID","raMean","decMean"])

## Initialize crossmatch and compute, measuring the time elapsed.

In [ ]:
# Setting up the cross-match actually takes very little time
ztf_x_ps1 = ztf_piece.crossmatch(ps1, radius_arcsec=1, n_neighbors=1, suffixes=("_ztf", "_ps1"))

In [ ]:
# Executing the cross-match does take time
%%time
xmatch = ztf_x_ps1.compute()

In [ ]:
# Check the length of the resulting table
len(xmatch)

In [ ]:
# For small enough cross-matches, it's useful to compute several times and take the average time
# ONLY USE FOR Nrows <= 1_000_000 or so 
%%timeit
xmatch = ztf_x_ps1.compute()

## Record benchmarks

Benchmarks on Fornax Large instance using no dask (t1) and autoscaling dask with 2-16 workers (t2).
"x" indicates out-of-memory behavior.

| N rows | t1 (s) | t2 (s) |
| ------ | ------ | ------ |
| 1e4    | 2.61   | 8.09   |
| 1e5    | 3.55   | 4.2    |
| 1e6    | 6.77   | 9.95   |
| 1e7    | 41.5   | x      |
| 1e8    | x      | x      |

Benchmarks on Fornax XLarge instance using no dask (t1) and autoscaling dask with 2-16 workers (t2).
"x" indicates out-of-memory behavior.

| N rows | t1 (s) | t2 (s) |
| ------ | ------ | ------ |
| 1e4    |  1.6   |    |
| 1e5    |  1.85  |    |
| 1e6    |  3.53  |    |
| 1e7    |  18.4  |    |
| 1e8    |  364   |    |